## Data Exploration and Cleaning of the NYC Department of Building Job Applications Filing Dataset

This notebook will explore some of the features of the dataset and discuss some of the logics that were used to develop the data cleaning methodology. The data cleaning is set-up in a way so machine learning can be applied to the dataset.

Source dataset found [here](https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2/)

In [1]:
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import re

from helper_scripts.cleaning_scripts import *

In [2]:
raw_data_path = os.path.join('..', 'data', 'raw', 'DOB_job_application.csv')
df = pd.read_csv(raw_data_path, low_memory=False)
df.head()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,Job Status Descrp,Latest Action Date,Building Type,Community - Board,Cluster,Landmarked,Adult Estab,Loft Board,City Owned,Little e,PC Filed,eFiling Filed,Plumbing,Mechanical,Boiler,Fuel Burning,Fuel Storage,Standpipe,Sprinkler,Fire Alarm,Equipment,Fire Suppression,Curb Cut,Other,Other Description,Applicant's First Name,Applicant's Last Name,Applicant Professional Title,Applicant License #,Professional Cert,Pre- Filing Date,Paid,Fully Paid,Assigned,Approved,Fully Permitted,Initial Cost,Total Est. Fee,Fee Status,Existing Zoning Sqft,Proposed Zoning Sqft,Horizontal Enlrgmt,Vertical Enlrgmt,Enlargement SQ Footage,Street Frontage,ExistingNo. of Stories,Proposed No. of Stories,Existing Height,Proposed Height,Existing Dwelling Units,Proposed Dwelling Units,Existing Occupancy,Proposed Occupancy,Site Fill,Zoning Dist1,Zoning Dist2,Zoning Dist3,Special District 1,Special District 2,Owner Type,Non-Profit,Owner's First Name,Owner's Last Name,Owner's Business Name,Owner's House Number,Owner'sHouse Street Name,City,State,Zip,Owner'sPhone #,Job Description,DOBRunDate,JOB_S1_NO,TOTAL_CONSTRUCTION_FLOOR_AREA,WITHDRAWAL_FLAG,SIGNOFF_DATE,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
0,420291794,1,QUEENS,183-16,JAMAICA AVENUE,10352,00058,4438800,A3,X,SIGNED OFF,08/02/2011 12:00:00 AM,OTHERS,412,N,N,N,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,ANTENNA,MANOJKUMAR,PATEL,PE,084829,N,11/29/2010,11/29/2010,12/29/2010,12/30/2010,01/10/2011,01/25/2011,$2000.00,$100.00,STANDARD,0,0,NaN,NaN,0,0,2,2,24,24,NaN,NaN,COM,COM,NaN,M1-2,NaN,NaN,NaN,IBZ,PARTNERSHIP,N,JOE,VIGNA,NEW CINGULAR,340,340 MT. KEMBEL,MORRISTOWN,NY,07960,9738964570,INSTALLING 2'X2' TELECOMMUNICATIONS EQUIPMENT ...,03/13/2019 12:00:00 AM,1841656,0,0,08/02/2011,N,NaN,E1,0,40.708932,-73.776983,27.0,444.0,Jamaica,4438800.0
1,420291801,1,QUEENS,150-33,CENTREVILLE STREET,11556,00053,4248973,A2,X,SIGNED OFF,10/31/2011 12:00:00 AM,1-2-3 FAMILY,410,N,N,N,N,NaN,N,NaN,Y,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANDREW,SAYERS,PE,077987,N,11/29/2010,11/29/2010,11/29/2010,11/30/2010,01/21/2011,01/21/2011,$15000.00,$151.50,STANDARD,0,0,NaN,NaN,0,0,3,3,35,35,2,2,R-3,R-3,NOT APPLICABLE,R4,NaN,NaN,NaN,NaN,CORPORATION,N,ANTHONY,GURINO,VMG REALITY LLC,96-20,96-20 ATLANTIC AVENUE,OZONE PARK,NY,11416,7186411098,INSTALLATION OF WET SPRINKLER SYSTEM OFF DOMES...,03/13/2019 12:00:00 AM,1841667,0,0,10/31/2011,N,NaN,B2,0,40.669071,-73.838928,32.0,88.0,Ozone Park,4541404.0
2,340644128,1,BROOKLYN,354,DOUGLAS STREET,00946,00015,3019327,A2,X,SIGNED OFF,03/18/2019 12:00:00 AM,1-2-3 FAMILY,306,N,N,N,N,NaN,N,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,SOLAR TAX,JAMES,CLANCY,PE,084288,N,11/02/2018,11/02/2018,11/02/2018,11/05/2018,11/05/2018,11/08/2018,$44726.00,$234.00,STANDARD,0,0,NaN,NaN,0,0,2,2,28,28,2,2,R-3,R-3,NOT APPLICABLE,R6B,NaN,NaN,NaN,NaN,INDIVIDUAL,N,MARK,CAVANAGH,NaN,354,354 DOUGLASS STREET,BROOKLYN,NY,11217,7188527096,INSTALL SOLAR PV SYSTEM ON THE EXISTING ROOFTO...,03/19/2019 12:00:00 AM,3026504,0,0,03/18/2019,N,NaN,B1,0,40.679058,-73.981126,39.0,131.0,Park Slope-Gowanus,3019327.0
3,421685439,1,QUEENS,34-56,200 STREET,06078,00031,4619372,NB,P,PLAN EXAM - APPROVED,04/09/2019 12:00:00 AM,1-2-3 FAMILY,411,N,N,N,N,NaN,N,NaN,Y,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,GEN. CONSTR.,DEZHANG,FANG,RA,037876,N,11/02/2018,11/02/2018,11/02/2018,11/05/2018,04/09/2019,NaN,$0.00,$243.00,STANDARD,0,2700,NaN,NaN,0,0,0,2,0,28,NaN,2,NaN,R-3,USE UNDER 300 CU.YD,R3A,NaN,NaN,NaN,NaN,PARTNERSHIP,N,ZE,CHEN,GREAT INVESTMENT USA LLC,42-26,42-26 161 STREET,FLUSHING,NY,11358,9172388321,HEREWITH FILED FOR A NEW DETACHED 2-STORY 2-FA...,04/10/2019 12:00:00 AM,3026432,4050,0,NaN,N,NaN,A0,0,40.765658,-73.787240,19.0,1099.0,Bayside-Bayside Hills,NaN
4,421677974,2,QUEENS,35-37,36TH STREET,00640,00004,4009599,A2,R,PERM

In [3]:
df.columns = [clean_col_name(x) for x in df.columns] ## clean-up column name: removes symbols, # --> Num, and replaces space with _
## Remove columns that will not be used for this analysis
exclude_columns = ['House_Num', 'Street_Name', 'Block', 'Lot', 'Bin_Num', 'Job_Status_Descrp', 'Latest_Action_Date', 'Applicants_First_Name', 'Applicants_Last_Name', 'Applicant_Professional_Title', 
                   'Applicant_License_Num','Professional_Cert', 'Owners_First_Name','Owners_Last_Name','Owners_Business_Name','Owners_House_Number','OwnersHouse_Street_Name',
                   'City','State','Zip','OwnersPhone_Num', 'DOBRunDate','JOB_S1_NO', 'GIS_COUNCIL_DISTRICT','GIS_CENSUS_TRACT', 'GIS_BIN']
df.drop(exclude_columns, inplace=True, axis=1)
## Convert columns that have dollar signs to float so it can be aggregated
df['Initial_Cost'], df['Total_Est_Fee'] = get_dollar_to_num(df['Initial_Cost']), get_dollar_to_num(df['Total_Est_Fee'])

df.head()

,Job_Num,Doc_Num,Borough,Job_Type,Job_Status,Building_Type,Community_Board,Cluster,Landmarked,Adult_Estab,Loft_Board,City_Owned,Little_e,PC_Filed,eFiling_Filed,Plumbing,Mechanical,Boiler,Fuel_Burning,Fuel_Storage,Standpipe,Sprinkler,Fire_Alarm,Equipment,Fire_Suppression,Curb_Cut,Other,Other_Description,Pre_Filing_Date,Paid,Fully_Paid,Assigned,Approved,Fully_Permitted,Initial_Cost,Total_Est_Fee,Fee_Status,Existing_Zoning_Sqft,Proposed_Zoning_Sqft,Horizontal_Enlrgmt,Vertical_Enlrgmt,Enlargement_SQ_Footage,Street_Frontage,ExistingNo_of_Stories,Proposed_No_of_Stories,Existing_Height,Proposed_Height,Existing_Dwelling_Units,Proposed_Dwelling_Units,Existing_Occupancy,Proposed_Occupancy,Site_Fill,Zoning_Dist1,Zoning_Dist2,Zoning_Dist3,Special_District_1,Special_District_2,Owner_Type,NonProfit,Job_Description,TOTAL_CONSTRUCTION_FLOOR_AREA,WITHDRAWAL_FLAG,SIGNOFF_DATE,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_NTA_NAME
0,420291794,1,QUEENS,A3,X,OTHERS,412,N,N,N,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,ANTENNA,11/29/2010,11/29/2010,12/29/2010,12/30/2010,01/10/2011,01/25/2011,2000.0,100.0,STANDARD,0,0,NaN,NaN,0,0,2,2,24,24,NaN,NaN,COM,COM,NaN,M1-2,NaN,NaN,NaN,IBZ,PARTNERSHIP,N,INSTALLING 2'X2' TELECOMMUNICATIONS EQUIPMENT ...,0,0,08/02/2011,N,NaN,E1,0,40.708932,-73.776983,Jamaica
1,420291801,1,QUEENS,A2,X,1-2-3 FAMILY,410,N,N,N,N,NaN,N,NaN,Y,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/29/2010,11/29/2010,11/29/2010,11/30/2010,01/21/2011,01/21/2011,15000.0,151.5,STANDARD,0,0,NaN,NaN,0,0,3,3,35,35,2,2,R-3,R-3,NOT APPLICABLE,R4,NaN,NaN,NaN,NaN,CORPORATION,N,INSTALLATION OF WET SPRINKLER SYSTEM OFF DOMES...,0,0,10/31/2011,N,NaN,B2,0,40.669071,-73.838928,Ozone Park
2,340644128,1,BROOKLYN,A2,X,1-2-3 FAMILY,306,N,N,N,N,NaN,N,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,SOLAR TAX,11/02/2018,11/02/2018,11/02/2018,11/05/2018,11/05/2018,11/08/2018,44726.0,234.0,STANDARD,0,0,NaN,NaN,0,0,2,2,28,28,2,2,R-3,R-3,NOT APPLICABLE,R6B,NaN,NaN,NaN,NaN,INDIVIDUAL,N,INSTALL SOLAR PV SYSTEM ON THE EXISTING ROOFTO...,0,0,03/18/2019,N,NaN,B1,0,40.679058,-73.981126,Park Slope-Gowanus
3,421685439,1,QUEENS,NB,P,1-2-3 FAMILY,411,N,N,N,N,NaN,N,NaN,Y,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,GEN. CONSTR.,11/02/2018,11/02/2018,11/02/2018,11/05/2018,04/09/2019,NaN,0.0,243.0,STANDARD,0,2700,NaN,NaN,0,0,0,2,0,28,NaN,2,NaN,R-3,USE UNDER 300 CU.YD,R3A,NaN,NaN,NaN,NaN,PARTNERSHIP,N,HEREWITH FILED FOR A NEW DETACHED 2-STORY 2-FA...,4050,0,NaN,N,NaN,A0,0,40.765658,-73.787240,Bayside-Bayside Hills
4,421677974,2,QUEENS,A2,R,OTHERS,401,N,N,NaN,N,NaN,NaN,NaN,Y,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/12/2018,09/12/2018,09/12/2018,NaN,NaN,NaN,105000.0,1275.6,STANDARD,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CORPORATION,N,MECHANICAL AND PLUMBING WORK AS SHOWN ON PLANS,0,0,NaN,N,NaN,E9,0,40.755622,-73.925003,Astoria


For a specific filing/job with the DOB there are multiple job permit applications filed. For example, in the job below there are 5 separate documents that were filed. Most fields are duplicated information across the document numbers, but some columns contain information that appear only in that particular document number. For example, columns like *Plumbing, Mechanical, Boiler*, etc. only have the X indicator if that document number that was associated with a Plumbing work filing. When aggregating the information we need to make sure these information are not lost.

Further, columns like *Other Description* and *Job Description* have unique detailed descriptions of the job across documents which could be utilized in a model.

There are also details that only seem to appear on the first document, e.g. columns like *Proposed_Zoning_Sqft*, *TOTAL_CONSTRUCTION_FLOOR_AREA*, etc. Any data processing to build features for models will need to take these features of the dataset into consideration.

In [4]:
df[df['Job_Num']==320595940].sort_values('Doc_Num')

,Job_Num,Doc_Num,Borough,Job_Type,Job_Status,Building_Type,Community_Board,Cluster,Landmarked,Adult_Estab,Loft_Board,City_Owned,Little_e,PC_Filed,eFiling_Filed,Plumbing,Mechanical,Boiler,Fuel_Burning,Fuel_Storage,Standpipe,Sprinkler,Fire_Alarm,Equipment,Fire_Suppression,Curb_Cut,Other,Other_Description,Pre_Filing_Date,Paid,Fully_Paid,Assigned,Approved,Fully_Permitted,Initial_Cost,Total_Est_Fee,Fee_Status,Existing_Zoning_Sqft,Proposed_Zoning_Sqft,Horizontal_Enlrgmt,Vertical_Enlrgmt,Enlargement_SQ_Footage,Street_Frontage,ExistingNo_of_Stories,Proposed_No_of_Stories,Existing_Height,Proposed_Height,Existing_Dwelling_Units,Proposed_Dwelling_Units,Existing_Occupancy,Proposed_Occupancy,Site_Fill,Zoning_Dist1,Zoning_Dist2,Zoning_Dist3,Special_District_1,Special_District_2,Owner_Type,NonProfit,Job_Description,TOTAL_CONSTRUCTION_FLOOR_AREA,WITHDRAWAL_FLAG,SIGNOFF_DATE,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_NTA_NAME
60692,320595940,1,BROOKLYN,NB,X,OTHERS,307,N,N,N,N,NaN,N,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,X,X,GEN. CONSTR.,03/31/2014,04/01/2014,04/01/2014,05/22/2018,10/19/2015,09/27/2016,0.0,7834.16,STANDARD,0,20987,NaN,NaN,0,0,0,6,0,65,NaN,74,NaN,R-1,ON-SITE,M1-2D,NaN,NaN,NaN,NaN,PARTNERSHIP,N,PROPOSED 6-STORY HOTEL,28716,2,01/03/2019,N,NaN,H3,0,40.657099,-74.002984,Sunset Park West
60697,320595940,2,BROOKLYN,NB,X,OTHERS,307,N,N,NaN,N,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,STRUCTURAL,09/03/2014,09/03/2014,09/03/2014,05/22/2018,NaN,NaN,0.0,0.00,STANDARD,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARTNERSHIP,N,STRUCTURAL WORK,0,0,01/03/2019,N,NaN,H3,0,40.657099,-74.002984,Sunset Park West
60698,320595940,3,BROOKLYN,NB,X,OTHERS,307,N,N,NaN,N,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,SOE,09/03/2014,09/03/2014,09/03/2014,05/22/2018,NaN,NaN,0.0,0.00,STANDARD,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARTNERSHIP,N,SHORING WORK FOR SUPPORT OF EXCAVATION.,0,0,01/03/2019,N,NaN,H3,0,40.657099,-74.002984,Sunset Park West
798418,320595940,4,BROOKLYN,NB,D,OTHERS,307,N,N,NaN,N,NaN,NaN,NaN,Y,X,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,02/04/2015,02/04/2015,02/04/2015,NaN,NaN,NaN,0.0,0.00,STANDARD,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDIVIDUAL,N,"PLUMBING, SPRINKLER & HVAC SYSTEMS",0,0,NaN,W,08/15/2015,NaN,0,40.657099,-74.002984,Sunset Park West
159,320595940,5,BROOKLYN,NB,X,OTHERS,307,N,N,NaN,N,NaN,NaN,NaN,Y,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/21/2015,08/21/2015,08/21/2015,05/22/2018,NaN,NaN,0.0,0.00,STANDARD,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDIVIDUAL,N,PLUMBING & HVAC SYSTEMS,0,0,01/03/2019,N,NaN,H3,0,40.657099,-74.002984,Sunset Park West


In [5]:
interim_data_path = os.path.join('..', 'data', 'interim', 'DOB_job_application.csv')
df.to_csv(interim_data_path, index=False)

## Data Exploration

The A2 permits (non-major alterations) seems to be ones that are filed the most. There seems to be a significant amount of A1 (major alterations) and NB (new building) filings. 

In [6]:
df[['Job_Type']].groupby('Job_Type').size().sort_values(ascending=False)

Job_Type
A2    1049908
A3     290252
A1     133536
NB     118522
DM      44622
PA      21022
SI       9889
SC       6617
dtype: int64

No surprises here that Manhattan and Brooklyn filings are the most.

In [7]:
df[['Borough']].groupby('Borough').size().sort_values(ascending=False)

Borough
MANHATTAN        743920
BROOKLYN         378727
QUEENS           325348
BRONX            133466
STATEN ISLAND     92907
dtype: int64

The other description columns contain useful information but could use a lot of cleaning up. For example, the term *GENERAL CONSTRUCTION* seems to be coming up in a variety of forms such as GC, GEN.CONSTR., GEN. CONSTR, etc.

In [8]:
df[['Other_Description']].groupby('Other_Description').size().sort_values(ascending=False)[0:100]

Other_Description
GEN. CONSTR.       269735
GC                 116239
GEN. CONSTR         87360
STRUCTURAL          54217
BPP                 45171
GEN CONST           42113
GEN.CONSTRUCTN.     26552
CONSTRUCTION        20462
FACADE              19630
SOLAR TAX           18180
NO WORK             15421
ANTENNA             14049
AWNING              13128
PARTITIONS          12774
G.C.                12045
GEN. CONST.         10414
ARCHITECTURAL       10305
SOLAR                8516
ENL                  6160
GEN CONSTR           5075
GEN CONST.           4796
FPP                  4713
ARCH                 4641
Structural           4402
GEN CONSTRUCT        3637
SOE                  3350
CONST.               3190
FOUNDATION           2923
G.C                  2708
CONSTR               2510
CONST                2495
GENERAL CONSTRU      2126
TEMP INSTALL         1988
INTERIOR DEMO        1790
PARTITION            1590
SHORING              1455
DUNNAGE              1454
structural          

## Data Cleaning

As noted in the above section document number one contains a lot of information that are not available in the other document versions. The next set of codes will identify those that have information that we want to retain.

In [9]:
## Filter out job# where there is no doc 1
df_doc_min = df[['Job_Num', 'Doc_Num']].groupby(['Job_Num']).min().reset_index()
df_doc_non1 = pd.merge(df, df_doc_min[df_doc_min['Doc_Num']!=1], on=['Job_Num'])
del(df_doc_min)

## Keep only non doc1 versions that do not contain relevant info
df_doc_non1 = df_doc_non1[(df_doc_non1['Proposed_Zoning_Sqft'] == 0) & (df_doc_non1['Proposed_No_of_Stories'] == 0) & (df_doc_non1['TOTAL_CONSTRUCTION_FLOOR_AREA'] == 0)][['Job_Num']]
df_clean = pd.merge(df, df_doc_non1, how='outer', indicator=True) ## Left join
del(df_doc_non1)
df_clean = df_clean[df_clean['_merge']=='left_only']
df_clean.drop('_merge', axis=1, inplace=True)

The next set of codes will convert the columns that only contain **X** to a numerical value so it can be aggregated.

In [10]:
filing_columns = ['Plumbing', 'Mechanical', 'Boiler', 'Fuel_Burning','Fuel_Storage','Standpipe','Sprinkler','Fire_Alarm','Equipment','Fire_Suppression','Curb_Cut','Other']
df_clean[filing_columns] = df_clean[filing_columns].fillna(0)
df_clean[filing_columns] = df_clean[filing_columns].replace('X', 1)
df_clean[filing_columns] = df_clean[filing_columns].replace('N', 0)
agg_columns = filing_columns + ['Initial_Cost', 'Total_Est_Fee']
df_filing_details = df_clean[['Job_Num'] + agg_columns].groupby('Job_Num').sum().reset_index()

Next combine the *Job_Description* columns as well as the cleaned *Other Description* column so they may be turned into useful features during the modeling phase.

In [11]:
## Combine feature rows across doc num into one row
df_clean['Other_Description_cleaned'] = get_clean_other_desc(df_clean['Other_Description'])
df_jobdesc = df_clean[~df_clean['Job_Description'].isna()][['Job_Num', 'Job_Description']].groupby('Job_Num')['Job_Description'].apply(list).reset_index()
df_other_desc = df_clean[~df_clean['Other_Description_cleaned'].isna()][['Job_Num', 'Other_Description_cleaned']].groupby('Job_Num')['Other_Description_cleaned'].apply(list).reset_index()
df_details = pd.merge(df_jobdesc,df_other_desc, on='Job_Num',how='outer')
df_details = pd.merge(df_details, df_filing_details, on='Job_Num',how='outer')
ignore_columns = agg_columns + ['Other_Description_cleaned', 'Job_Description', 'Other_Description']
del(df_jobdesc, df_other_desc, df_filing_details) ## Delete to free up space

Combine all of the datasets and output

In [12]:
colnames = [x for x in df_clean.columns if x not in ignore_columns]
df_proc = df_clean[(~df_clean['Fully_Permitted'].isna()) & (~df_clean['Pre_Filing_Date'].isna()) & (df_clean['Doc_Num']==1)][colnames]
df_proc = pd.merge(df_proc, df_details, on='Job_Num')

processed_data_path = os.path.join('..', 'data', 'processed', 'DOB_job_application.csv')
df_proc.to_csv(processed_data_path, index=False)